In [23]:
# !pip install html5lib
# !pip install lxml 

In [1]:
import requests 
from bs4 import BeautifulSoup
import html
import pandas as pd, numpy as np 
from io import StringIO
import math  
import sqlite3, sqlalchemy 
import re 

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 

In [4]:
URL = 'https://en.wikipedia.org/wiki/List_of_NBA_All-Stars' 
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html') 
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-not-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of NBA All-Stars - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pin

In [2]:
def replace_status(s): 
    if pd.isnull(s):
        return "Regular Selection" 
    
    return s 

def clean_player(s): 
    symbols = ['*', '^', '['] 
    i = -1  
    for symbol in symbols:
        i = s.find(symbol) 
        if i > -1:
            return s[:i] 
 
    return s 

def season_split(Season): 
    Season = Season.replace('–', '-')
    split = Season.split(';') 
    new = [] 
    for i in range(len(split)): 
        if '-' in split[i]: 
            split[i] = split[i].split('-') 
            temp = [] 
            for j in range(len(split[i])):
                split[i][j] = int(split[i][j]) 
            for year in range(split[i][0],split[i][1]+1):
                season = str(year-1) + '-' + str(year)[2:]
                split[i].append(season)
            split[i].pop(0) 
            split[i].pop(0)
        else: 
            year = int(split[i].strip())
            split[i] = [str(year-1) + '-' + str(year)[2:]]
        new.extend(split[i])

    return np.concatenate(split) 

def clean_status(status): 
    key_words = ['captain', 'missed', 'replacement', 'mvp']
    # status = status.replace(' ', '').lower() 
    status = re.sub('\[\d+\]', '', status).lower() 
    status = re.sub(',', '', status).lower() 
    status = re.sub('\.', '', status).lower() 

    split = status.split(' ')
    token = '' 
    return_status = [] 

    for i in range(len(split)): 
        if split[i] in key_words: 
            token = split[i] 
        elif (re.compile('\d{4}').search(split[i])) is not None: 
            temp = int(split[i]) 

            return_status.append(token + ' ' + str(temp-1) + '-' + str(temp)[2:])


    # if token == '':
    #     split = "Regular" 

    return return_status 


def select_status(row): 
    # print(row)
    season = row['Season'] 
    season_status = 'regular' 
    for status in row['Status']: 
        if season in status: 
            season_status = re.sub('[\d -]*', '', status)  
    return season_status 
                                

<>:42: SyntaxWarning: invalid escape sequence '\['
<>:44: SyntaxWarning: invalid escape sequence '\.'
<>:53: SyntaxWarning: invalid escape sequence '\d'
<>:71: SyntaxWarning: invalid escape sequence '\d'
<>:42: SyntaxWarning: invalid escape sequence '\['
<>:44: SyntaxWarning: invalid escape sequence '\.'
<>:53: SyntaxWarning: invalid escape sequence '\d'
<>:71: SyntaxWarning: invalid escape sequence '\d'
C:\Users\bchan\AppData\Local\Temp\ipykernel_16748\413249341.py:42: SyntaxWarning: invalid escape sequence '\['
  status = re.sub('\[\d+\]', '', status).lower()
C:\Users\bchan\AppData\Local\Temp\ipykernel_16748\413249341.py:44: SyntaxWarning: invalid escape sequence '\.'
  status = re.sub('\.', '', status).lower()
C:\Users\bchan\AppData\Local\Temp\ipykernel_16748\413249341.py:53: SyntaxWarning: invalid escape sequence '\d'
  elif (re.compile('\d{4}').search(split[i])) is not None:
C:\Users\bchan\AppData\Local\Temp\ipykernel_16748\413249341.py:71: SyntaxWarning: invalid escape sequence '

In [8]:
allstars = soup.find('table', attrs={'class':'wikitable sortable'})
allstars = pd.read_html(StringIO(str(allstars)), skiprows=0) 
allstars = pd.DataFrame(allstars[0])
allstars = allstars[['Player', 'Selections[c]', 'Notes']] 
allstars = allstars.rename(columns={'Selections[c]':'Season', 'Notes':'Status'}) 
allstars['Status'] = allstars['Status'].apply(lambda x: 'Regular' if pd.isnull(x) else x) 
allstars['Player'] = allstars['Player'].apply(lambda x: clean_player(x)) 
allstars['Season'] = allstars['Season'].apply(lambda x: season_split(x)) 
allstars = allstars.explode('Season').reset_index().drop('index', axis=1) 
allstars['Status'] = allstars['Status'].apply(lambda x: clean_status(x)) 
allstars['Status'] = allstars.apply(lambda x: select_status(x), axis=1) 


In [19]:
# allstars[allstars['Season']=='2004-05'] 
allstars[(allstars['Player']=='LeBron James') & (allstars['Season']=='2004-05')]
# allstars['Season'].dtype 

,Player,Season,Status
0,LeBron James,2004-05,regular


In [28]:
engine = sqlalchemy.create_engine("sqlite:///C:\\Users\\bchan\\OneDrive\\Personal Projects\\BID_Django\\ballindata\\DB\\ballbase.db") 
 
allstars.to_sql('allstars', con=engine, if_exists="replace", index=False)

1795